# Overview
We wish to know the excess electricity production capacity in every state in the U.S. during a specific month. The idea is to find out how well states are poised to serve excess demand during extreme weather events. For example, if we find out the excess capacity of a state in july, it might indicate how well the state can handle spikes in demand during heat waves. Similarly, if excess capacity of a state in january might indicate how well it can handle cold waves. 

# Links
  1. [EIA - Statewise generation capacity](https://www.eia.gov/opendata/browser/electricity/state-electricity-profiles/capability?frequency=annual&data=capability;&start=2022&end=2022&sortColumn=period;&sortDirection=desc;)
  2. [EIA - Regional, statewise electricity consumption](https://www.eia.gov/opendata/browser/electricity/retail-sales?frequency=monthly&data=sales;&start=2024-01&end=2024-01&sortColumn=period;&sortDirection=desc;)

In [30]:
import pandas as pd
import requests
import json
import sys
sys.path.append("../")
from eia_utils import get_state_electricity_capacity, get_state_electricity_consumption

In [31]:
# Obtain the power production capability data from EIA database
raw_data = get_state_electricity_capacity("2022", "2022")

In [32]:
print("Raw data shape and size")
print(raw_data.size)
print(raw_data.shape)

# Fix the data types right and explore the capabilities data
raw_data['capability'] = raw_data['capability'].astype('float')
print("Sample capabilities data")
print(raw_data.head())
print("Unique state values")
print(raw_data['stateDescription'].unique())
print("No. of unique state values: ", len(raw_data['stateDescription'].unique()))
print("Unique energy sources: ") 
print(raw_data['energySourceDescription'].unique())
print('Sanity check: Make sure there are not mixed units')
print(raw_data['capability-units'].unique())



Raw data shape and size
21141
(2349, 9)
Sample capabilities data
  period stateId stateDescription producertypeid      producerTypeDescription  \
0   2022      MS      Mississippi            IPP  Independent Power Producers   
1   2022      MS      Mississippi            IPP  Independent Power Producers   
2   2022      MS      Mississippi            IPP  Independent Power Producers   
3   2022      MS      Mississippi            TOT                  All sectors   
4   2022      MS      Mississippi            TOT                  All sectors   

  energysourceid energySourceDescription  capability capability-units  
0            SOL                   Solar       219.3        megawatts  
1          SOLPV              Solar - PV       219.3        megawatts  
2            WOO                    Wood       300.8        megawatts  
3            ALL                     All     14723.5        megawatts  
4            COL                    Coal      1444.0        megawatts  
Unique state val

While calculating the excess power capacity of a state if we are excluding wind energy for both the regular risk rating as well as the green risk rating. The reason is simple: When a heat wave occurs, it happens so because the absence of wind causes heat to stay in one place (instead of being dispersed by wind). So, by design, wind turbines won't be generating any power during heat waves. 

In [33]:
nonwind_raw_data = raw_data[~raw_data['energySourceDescription'].isin(['Wind'])]
green_raw_data = raw_data[raw_data['energySourceDescription'].isin(['Solar', 'Solar - PV', 
                                                                    'Hydroelectric', 'Geothermal', 'Solar - TH', 'Nuclear'])]
# Fix the data types right and explore the capabilities data
green_raw_data['capability'] = green_raw_data['capability'].astype('float')
print("Sample capabilities data")
print(green_raw_data.head())
print("Unique green energy sources: ") 
print(green_raw_data['energySourceDescription'].unique())
print('Sanity check: Make sure there are not mixed units')


Sample capabilities data
   period stateId stateDescription producertypeid  \
0    2022      MS      Mississippi            IPP   
1    2022      MS      Mississippi            IPP   
10   2022      MS      Mississippi            TOT   
15   2022      MS      Mississippi            TOT   
16   2022      MS      Mississippi            TOT   

        producerTypeDescription energysourceid energySourceDescription  \
0   Independent Power Producers            SOL                   Solar   
1   Independent Power Producers          SOLPV              Solar - PV   
10                  All sectors            NUC                 Nuclear   
15                  All sectors            SOL                   Solar   
16                  All sectors          SOLPV              Solar - PV   

    capability capability-units  
0        219.3        megawatts  
1        219.3        megawatts  
10      1400.5        megawatts  
15       319.3        megawatts  
16       319.3        megawatts  
Unique 

C:\Users\dinesh\AppData\Local\Temp\ipykernel_16480\1644171281.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  green_raw_data['capability'] = green_raw_data['capability'].astype('float')


In [34]:
# Sum up capability for each state and remove data related to stateDescription of "United States" and "District of Columbia"
capability_df = nonwind_raw_data.groupby('stateDescription')[['capability']].sum()
capability_df.drop(['United States', 'District of Columbia'], inplace=True, errors='ignore')
capability_df['capability-units'] = nonwind_raw_data.iloc[0]['capability-units']
print(capability_df.head())

                  capability capability-units
stateDescription                             
Alabama             145267.4        megawatts
Alaska               15192.4        megawatts
Arizona             146860.4        megawatts
Arkansas             72772.4        megawatts
California          442084.6        megawatts


In [35]:
# Sum up capability for each state and remove data related to stateDescription of "United States" and "District of Columbia"
green_capability_df = green_raw_data.groupby('stateDescription')[['capability']].sum()
green_capability_df.drop(['United States', 'District of Columbia'], inplace=True)
green_capability_df['capability-units'] = raw_data.iloc[0]['capability-units']
print(green_capability_df.head())

                  capability capability-units
stateDescription                             
Alabama              19173.4        megawatts
Alaska                 955.2        megawatts
Arizona              25310.2        megawatts
Arkansas              7493.2        megawatts
California           98944.6        megawatts


In [36]:
# Get the monthly consumption data from EIA. We assume the month of july would be the highest due to summer heat
# TODO: This assumption, while fair, should be done away with. We should look at all the months for every state and choose the summer month where the electricity usage is the highest for that state
raw_data = get_state_electricity_consumption("2023-07", "2023-07")


In [37]:

# Fix data types and explore the consumption data
raw_data['sales'] = raw_data['sales'].astype('float')
print("Sample consumption data")
print(raw_data.head())
print("Unique state values")
print(raw_data['stateDescription'].unique())
print("No. of unique state values: ", len(raw_data['stateDescription'].unique()))
print('Sanity check: Make sure there are not mixed units')
print(raw_data['sales-units'].unique())

Sample consumption data
    period stateid       stateDescription sectorid      sectorName  \
0  2023-07    PACC     Pacific Contiguous      RES     residential   
1  2023-07    PACC     Pacific Contiguous      TRA  transportation   
2  2023-07    PACN  Pacific Noncontiguous      ALL     all sectors   
3  2023-07    PACN  Pacific Noncontiguous      COM      commercial   
4  2023-07    PACN  Pacific Noncontiguous      IND      industrial   

         sales            sales-units  
0  12698.66275  million kilowatthours  
1     72.43836  million kilowatthours  
2   1249.70788  million kilowatthours  
3    448.67737  million kilowatthours  
4    425.51939  million kilowatthours  
Unique state values
['Pacific Contiguous' 'Pacific Noncontiguous' 'U.S. Total' 'Ohio'
 'Maryland' 'Massachusetts' 'Michigan' 'Minnesota' 'Mississippi'
 'Missouri' 'Montana' 'Nebraska' 'Nevada' 'New Hampshire' 'New Jersey'
 'New Mexico' 'New York' 'North Carolina' 'North Dakota' 'Alabama'
 'Alaska' 'Arizona' 'Arkan

In [38]:
states_only_consumption_df = raw_data[raw_data['stateDescription'].isin(capability_df.index)]
monthly_consumption_df = states_only_consumption_df.groupby('stateDescription')[['sales']].sum()
monthly_consumption_df['sales-units'] = states_only_consumption_df.iloc[0]['sales-units']
print("Monthly consumption data sample:")
print(monthly_consumption_df.head())
print("Sanity check: Ensure there are 50 and only 50 states in the monthly consumption dataframe:")
print("No. of states for which consumption data is available: ", len(monthly_consumption_df.index))

Monthly consumption data sample:
                        sales            sales-units
stateDescription                                    
Alabama           17218.67582  million kilowatthours
Alaska              941.50372  million kilowatthours
Arizona           21346.60244  million kilowatthours
Arkansas           9704.07617  million kilowatthours
California        45881.17636  million kilowatthours
Sanity check: Ensure there are 50 and only 50 states in the monthly consumption dataframe:
No. of states for which consumption data is available:  50


## Excess capacity calculation
The production capability data (*capability_df*) mentions power capability in megawatts. To calculate excess capacity, we need to find to first out the average power consumption in mega watts given the monthly energy consumption - i.e., sales - figure (*monthly_consumption_df*). To do this, we go through the following steps:

  1. Convert sales in million kilowatthours to kilowatthours:
  $$
  sales(kWh/mo.) = sales(million\ kWh/mo.)*1000000
  $$
  2. Convert sales in kilowatthours into megawatthours
  $$
  sales(MWh/mo.) = \dfrac{sales(kWh/mo.)}{1000}
  $$
  3. Deduce the average power consumption in MW throughout the month by dividing the energy consumption in MWh by divided it with the number of hours in a month
  $$
  Avg.\ power\ consumption (MW) =  \dfrac{sales(MWh/mo.)}{30(days/mo.)*24(hours/day)} = \dfrac{sales(MWh/mo.)}{30*24(hours/mo.)}
  $$

After obtaining the average power consumption in mega watts, we need to one more thing: From EIA [gid monitor](https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/US48/US48) data, we find that the day time peak in the month of july is 1.25 times the average power consumption during the day (I measured it manually by eyeballing the graphs). We can combine the three steps described above and the scaling of 1.25 in one step as:
$$
Peak\ power\ consumption (MW) = \dfrac{sales(million\ kWh/mo.)*1250}{30*24}
$$

In [39]:
# Calculate excess capacity
# First reverse engineer state hourly consumption level and find the max hourly consumption 
excess_capacity =  capability_df['capability'] - monthly_consumption_df['sales']*1250/(30*24)
excess_capacity_df = pd.DataFrame(excess_capacity, columns=['excess capacity'])
excess_capacity_df.reset_index(inplace=True)
print(excess_capacity_df)
excess_capacity_df.to_csv('excess_capacity.csv', index=False)

   stateDescription  excess capacity
0           Alabama    115373.865590
1            Alaska     13557.844931
2           Arizona    109800.326319
3          Arkansas     55925.045538
4        California    362429.779931
5          Colorado     58860.889549
6       Connecticut     45459.757743
7          Delaware     14880.623715
8           Florida    290373.086042
9           Georgia    136205.203542
10           Hawaii     13066.491632
11            Idaho     12818.496771
12         Illinois    154080.808854
13          Indiana     86109.273924
14             Iowa     58276.576701
15           Kansas     51236.343090
16         Kentucky     61600.223750
17        Louisiana    104235.214497
18            Maine     20861.391580
19         Maryland     41906.055590
20    Massachusetts     53858.718437
21         Michigan    111810.049653
22        Minnesota     58409.283941
23      Mississippi     64535.448125
24         Missouri     66010.438854
25          Montana     19331.269097
2

In [40]:
# Calculate green excess capacity
# First reverse engineer state hourly consumption level and find the max hourly consumption 
green_excess_capacity =  green_capability_df['capability'] - monthly_consumption_df['sales']*1250/(30*24)
green_excess_capacity_df = pd.DataFrame(green_excess_capacity, columns=['green excess capacity'])
green_excess_capacity_df.reset_index(inplace=True)
print(green_excess_capacity_df)
green_excess_capacity_df.to_csv('green_excess_capacity.csv', index=False)

   stateDescription  green excess capacity
0           Alabama          -10720.134410
1            Alaska            -679.355069
2           Arizona          -11749.873681
3          Arkansas           -9354.154462
4        California           19289.779931
5          Colorado          -12306.310451
6       Connecticut           -4829.042257
7          Delaware           -3564.576285
8           Florida          -58995.313958
9           Georgia          -24736.196458
10           Hawaii           -1245.508368
11            Idaho           -3067.103229
12         Illinois          -18520.991146
13          Indiana          -29526.926076
14             Iowa          -15422.223299
15           Kansas          -12234.456910
16         Kentucky          -20829.176250
17        Louisiana          -28313.585503
18            Maine            -570.808420
19         Maryland          -13726.944410
20    Massachusetts          -11544.881563
21         Michigan          -22860.550347
22        M

In [41]:
# Data visualization
